In [ ]:
pip install catboost

     |████████████████████████████████| 67.3MB 54kB/s 


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from tqdm import tqdm
import warnings
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error 
from sklearn import preprocessing
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/train.csv',  encoding='ISO-8859-1')
df_test = pd.read_csv('/content/drive/MyDrive/test.csv',  encoding='ISO-8859-1')

In [ ]:
df_train.head()

,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0


In [ ]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [ ]:
df = pd.concat([df_train, df_test])

In [ ]:
df = df.reset_index()
df.head()

,index,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
def remove_stop_words(sir):
    splited = sir.split(' ')
    splited = [word for word in splited if word not in stopwords]
    return ' '.join(splited)

df['Review_Text'] = df['Review_Text'].apply(remove_stop_words)

In [ ]:
expr = r'[^\w ]'
parser=re.compile(expr)
df['Review_Text'] = list(map(lambda x: (re.sub(expr, '', x)).split(), df['Review_Text']))

In [ ]:
dictionary = []

for d in tqdm(df['Review_Text']):
    dictionary = np.concatenate((dictionary, np.unique(d)))

100%|██████████| 4279/4279 [01:47<00:00, 39.93it/s]


In [ ]:
dictionary = np.unique(dictionary)

In [ ]:
dictionary = list(dictionary)

In [ ]:
from nltk import wordnet, pos_tag
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk import WordNetLemmatizer
def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])

In [ ]:
dictionary_lem = np.unique((my_lemmatizer(' '.join(dictionary))).split())

In [ ]:
df_review_text_train = []
for sentence in tqdm(df_train['Review_Text']):
        sentence = list(map(lambda x: ' '.join([word for word in x.split() if not word in stopwords]), sentence.split()))
        sentence = list(map(lambda x: re.sub(expr, '', x), sentence))
        spisok = []
        for sent in sentence:
            if sent != '':
                spisok.append(my_lemmatizer(sent))
        df_review_text_train.append(spisok)

100%|██████████| 2136/2136 [00:08<00:00, 247.50it/s]


In [ ]:
df_review_text_test = []
for sentence in tqdm(df_test['Review_Text']):
        sentence = list(map(lambda x: ' '.join([word for word in x.split() if not word in stopwords]), sentence.split()))
        sentence = list(map(lambda x: re.sub(expr, '', x), sentence))
        spisok = []
        for sent in sentence:
            if sent != '':
                spisok.append(my_lemmatizer(sent))
        df_review_text_test.append(spisok)

100%|██████████| 2143/2143 [00:08<00:00, 250.05it/s]


In [ ]:
df_lem = df_review_text_train + df_review_text_test

In [ ]:
df['Review_Text'] = df_lem

One hot encoding

In [ ]:
df[dictionary_lem] = 0

In [ ]:
df.head(3)

,index,Id,Hotel_name,Review_Title,Review_Text,Rating,0,00,001,01,0430,0500,0930am,1,10,100,1000,1000pm,100100,100716,100ft,100kms,100meters,1010,1015pm,10am,10pm,10th,11,110,1100,111,111111111111111E249,111130pm,115,1150,11am,11pm,11th,12,...,worthless,worthy,would,wow,write,wrong,wry,wud,x,xxxxxx,xxxxxxx,xxxxxxxxxxxxxxxxxxx,yaatra,yacht,yaha,yard,yaru,yeah,year,yellow,yet,ygyugiguug,yhgktrh,yoga,you,young,youre,yourself,yourselves,yrs,ysbut,yukkk,yum,yummacious,yummy,yyy66,zero,zinc,zone,zoo
0,0,0,Park Hyatt,Refuge in Chennai,"[Excellent, room, exercise, facility, All, aro...",80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,Hilton Chennai,Hilton Chennai,"[Very, comfortable, felt, safe, Staff, helpful...",100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,The Royal Regency,No worth the rating shown in websites. Pricing...,"[Not, worth, rating, show, Service, good, Room...",71.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
columns = list(df.columns)[6:]
for i in tqdm(range(len(df))):
    for j in range(len(columns)):
        if columns[j] in df['Review_Text'][i]:
            df[columns[j]][i] += 1

100%|██████████| 4279/4279 [04:07<00:00, 17.26it/s]


In [ ]:
df = df.drop(['index', 'Id'], axis=1)

In [ ]:
df_train = df[:2136]
df_test = df[2136:]

In [ ]:
def encoder(df, column):
    le = preprocessing.LabelEncoder()
    le.fit(list(df[column]))
    le.transform(list(df[column]))
    return pd.DataFrame(le.transform(list(df[column])))

In [ ]:
df_train['hotel_name'] = encoder(df_train, 'Hotel_name')

In [ ]:
df_train = df_train.drop(['Review_Title', 'Review_Text', 'Hotel_name'], axis=1)

In [ ]:
X = df_train.drop('Rating', axis=1)
y = df_train['Rating']

Считаем MSE на разных моделях

In [ ]:
cvs_rf = -1*np.mean(cross_val_score(RandomForestRegressor(), X, y, scoring='neg_mean_squared_error', cv=5))
print(f'RandomForestRegressor MSE: {cvs_rf}')

RandomForestRegressor MSE: 222.71762850993622


In [ ]:
cvs_cb = -1*np.mean(cross_val_score(CatBoostRegressor(silent=True), X, y, scoring='neg_mean_squared_error', cv=5))
print(f'CatBoostRegressor MSE: {cvs_cb}')

CatBoostRegressor MSE: 231.83991638059496


In [ ]:
cvs_lgbm = -1*np.mean(cross_val_score(LGBMRegressor(), X, y, scoring='neg_mean_squared_error', cv=7))
print(f'LGBMRegressor MSE: {cvs_lgbm}')

LGBMRegressor MSE: 232.6999262761887


Лучший результат показал RandomForestRegressor (MSE = 222.72)